## Exercise 2: MDSS Classifier

Detecting unfairness instances in subpopulations.

First, let's include the demo preprocessing to use the identified subgroups in the analysis

In [184]:
import itertools
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from aif360.metrics import BinaryLabelDatasetMetric, MDSSClassificationMetric
from aif360.detectors import bias_scan
from aif360.datasets import StandardDataset
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_compas

dataset_orig = load_preproc_data_compas()

female_group = [{'sex': 1}]
male_group = [{'sex': 0}]

dataset_orig_df = pd.DataFrame(dataset_orig.features, columns=dataset_orig.feature_names)

# Changes from one-hot encoding to categorical features
age_cat = np.argmax(dataset_orig_df[['age_cat=Less than 25', 'age_cat=25 to 45',
                                     'age_cat=Greater than 45']].values, axis=1).reshape(-1, 1)
priors_count = np.argmax(dataset_orig_df[['priors_count=0', 'priors_count=1 to 3',
                                          'priors_count=More than 3']].values, axis=1).reshape(-1, 1)
c_charge_degree = np.argmax(dataset_orig_df[['c_charge_degree=M', 'c_charge_degree=F']].values, axis=1).reshape(-1, 1)

features = np.concatenate((dataset_orig_df[['sex', 'race']].values, age_cat, priors_count,
                           c_charge_degree, dataset_orig.labels), axis=1)
feature_names = ['sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']

# Creates a structured dataset with new format to train a logistic regression classifier
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
dataset = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race'],
                 privileged_classes=[[1], [1]],
                 instance_weights_name=None)

dataset_orig_train, dataset_orig_test = dataset.split([0.7], shuffle=True, seed=0)

clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

# Makes predictions on test set
dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels

# Input probability predictions to determine privileged and unprivileged subsets
df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)

privileged_subset = bias_scan(df.iloc[:, :-2], df.observed, df.probabilities,
                              favorable_value=dataset_orig_test.favorable_label,
                              penalty=0.5, overpredicted=True)
unprivileged_subset = bias_scan(df.iloc[:, :-2], df.observed, df.probabilities,
                                favorable_value=dataset_orig_test.favorable_label,
                                penalty=0.5, overpredicted=False)
print(privileged_subset)
print(unprivileged_subset)

({'race': [0.0], 'age_cat': [0.0], 'sex': [0.0]}, 3.1526)
({'sex': [1.0], 'race': [0.0]}, 3.3036)


The MDSS demostration detects two subrgroups among the dataset that are biased beyond the other subgroups:
- 'race' = 0 (Not caucasian) + 'age_cat' = 0 (Less than 25) + 'sex' = 0 (Male)
- 'race' = 0 (Not caucasian) + 'sex' = 1 (Female)

The logistic regression model systematically underestimates the recidivism risk of individuals in the `Non-caucasian`, `less than 25`, `Male` subgroup whereas individuals belonging to the `Non-caucasian`, `Female` are assigned a higher risk than is actually observed. We refer to these subgroups as the `detected privileged group` and `detected unprivileged group` respectively.

Let's now compare this detected groups with their counterparts using MDSS to analyze the different hypothesis.

We create groups considering opposite 'race' and 'sex':

In [185]:
# Detected privileged group (group 1) and opposite group (group 2)
group_1 = [{'sex': 0, 'race': 0, 'age_cat': 0}]
group_2 = [{'sex': 1, 'race': 1, 'age_cat': 0}]

# Detected unprivileged group (group 3) and opposite group (group 4)
group_3 = [{'sex': 1, 'race': 0}]
group_4 = [{'sex': 0, 'race': 1}]

### 1st case: detected privileged group ('sex', 'race', and 'age_cat')

To be able to analyze through the MDSS classifier, we first need to create a new standard dataset with 'age_cat' also as a protected attribute.

In [186]:
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
dataset_pg = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race', 'age_cat'],
                 privileged_classes=[[1], [1], [0]],
                 instance_weights_name=None)

dataset_orig_train, dataset_orig_test = dataset_pg.split([0.7], shuffle=True, seed=0)

metric_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=group_2,
                             privileged_groups=group_1)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test.mean_difference())

Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.449811


Before using the classifier, and performing a simple mean difference bias, we can see that there is a considerable difference between the defined groups.

Let's now train a logistic regression model, make predictions and build a bias dataset:

In [187]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)

dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels

With this in hand, we can now perform the MDSS classification metric.

The idea of the method is to build apriori hypothesis. The MDSS bias will tell whether there is evidence to say that this logical statements have some fundamental basis.

Let's see our first example:
- We state that the unprivileged group is group 1 and that the privileged group is group 2

In [188]:
# Case 1
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=group_1,
                                           privileged_groups=group_2)
# Is there evidence that the hypothesized privileged group is actually privileged:
group_2_privileged_score = mdss_classified.score_groups(privileged=True)
print(group_2_privileged_score)
# Is there evidence that the hypothesized unprivileged group is actually unprivileged?
group_1_unprivileged_score = mdss_classified.score_groups(privileged=False)
print(group_1_unprivileged_score)

-0.0
-0.0


With these results, we can say the following:
- There is no evidence to say that group 2 (Caucasian females below 25 y.o.) is privileged (0.0)
- There is no evidence to say that group 1 (Non-caucasian males below 25 y.o.) is unprivileged (0.0)

Now we do the opposite scenario:

In [189]:
# Case 2
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=group_2,
                                           privileged_groups=group_1)
# Is there evidence that the hypothesized privileged group is actually privileged
group_1_privileged_score = mdss_classified.score_groups(privileged=True)
print(group_1_privileged_score)
# Is there evidence that the hypothesized unprivileged group is actually unprivileged
group_2_unprivileged_score = mdss_classified.score_groups(privileged=False)
print(group_2_unprivileged_score)

4.6526
2.2644


With the results of the second case, we can say the following:
- There is evidence to say that `group 1` (Non-caucasian males below 25 y.o.) `is privileged` (4.6526)
- There is evidence to say that `group 2` (Caucasian females below 25 y.o.) `is unprivileged` (2.2644)

These conclusions make sense considering that the group 1 subset was actually considered to be the most privileged group.

### 2nd case: detected unprivileged group ('sex' and 'race')

We perform the same steps for the detected unprivileged group

In [190]:
df = pd.DataFrame(features, columns=feature_names + ['two_year_recid'])
dataset_ug = StandardDataset(df, label_name='two_year_recid', favorable_classes=[0],
                 protected_attribute_names=['sex', 'race'],
                 privileged_classes=[[1], [1]],
                 instance_weights_name=None)

dataset_orig_train, dataset_orig_test = dataset_ug.split([0.7], shuffle=True, seed=0)

# Privileged group (group 4) has lower observed recidivism than group 3
metric_test = BinaryLabelDatasetMetric(dataset_orig_test,
                             unprivileged_groups=group_3,
                             privileged_groups=group_4)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test.mean_difference())

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2', random_state=0)
clf.fit(dataset_orig_train.features, dataset_orig_train.labels.flatten())

dataset_bias_test_prob = clf.predict_proba(dataset_orig_test.features)[:, 0]

df = pd.DataFrame(dataset_orig_test.features, columns=dataset_orig_test.feature_names)
df['observed'] = pd.Series(dataset_orig_test.labels.flatten(), index=df.index)
df['probabilities'] = pd.Series(dataset_bias_test_prob, index=df.index)

dataset_bias_test = dataset_orig_test.copy()
dataset_bias_test.scores = dataset_bias_test_prob
dataset_bias_test.labels = dataset_orig_test.labels

# Case 1
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=group_4,
                                           privileged_groups=group_3)
# Is there evidence that the hypothesized privileged group is actually privileged:
group_3_privileged_score = mdss_classified.score_groups(privileged=True)
print(group_3_privileged_score)
# Is there evidence that the hypothesized unprivileged group is actually unprivileged?
group_4_unprivileged_score = mdss_classified.score_groups(privileged=False)
print(group_4_unprivileged_score)

# Case 2
mdss_classified = MDSSClassificationMetric(dataset_orig_test, dataset_bias_test,
                                           unprivileged_groups=group_3,
                                           privileged_groups=group_4)
# Is there evidence that the hypothesized privileged group is actually privileged
group_4_privileged_score = mdss_classified.score_groups(privileged=True)
print(group_4_privileged_score)
# Is there evidence that the hypothesized unprivileged group is actually unprivileged
group_3_unprivileged_score = mdss_classified.score_groups(privileged=False)
print(group_3_unprivileged_score)


Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.069886
-0.0
0.263
-0.0
4.3036


By taking into account the size of the group and the magnitude of the deviation, mdss bias core has been able to tell us the following about the results:
- There is no evidence to say that group 3 (Non-caucasian females) is privileged (0.0)
- There is evidence to say that `group 4` (Caucasian males) `is unprivileged` (0.263)
- There is no evidence to say that group 4 (Caucasian males) is privileged (0.0)
- There is evidence to say that `group 3` (Non-caucasian females) `is unprivileged` (4.3036)

Contrary to the other subgroup analysis, here we have evidence to say that both groups are unprivileged. However, its also important to look at the bias metric because considering this analysis, there is bias evidence to say that the group 3 is more unprivileged than the also unprivileged group 4.

If we look at the mean difference collected before performing, we can see the number isn't far from 0, meaning that difference between defined groups isn't as big (considering the formerly analyzed group 1 and group 2)